In [6]:
# Chạy cell này đầu tiên
from bs4 import BeautifulSoup
from IPython.display import clear_output
import requests
import json
import os
import pandas as pd

In [7]:
# Merge file
# Không cần nhạy cell này
'''
from unidecode import unidecode

json_files = {
    "VN": "artist_VietNam.json", 
    "JP": "artist_Japan.json", 
    "EU": "artist_EU.json",
    "USUK": "artist_USUK.json",
    "KR": "artist_Korea.json",
    "Latin": "artist_Latin.json"
}


def normalize_name(name):
    return unidecode(name.lower())

def filter_similar_names(names):
    unique_names = set()
    filtered_names = []
    for name in names:
        normalized_name = normalize_name(name)
        if normalized_name not in unique_names:
            unique_names.add(normalized_name)
            filtered_names.append(name)
    return filtered_names

# Initialize an empty dictionary to store names and areas
name_area_dict = {}

# Loop through each JSON file
for area, json_file in json_files.items():
    with open(f"./{json_file}", 'r') as file:
        names = json.load(file)
        filtered_names = filter_similar_names(names)
        for name in filtered_names:
            if name in name_area_dict:
                name_area_dict[name] += f';{area}'
            else:
                name_area_dict[name] = area

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(name_area_dict.items()), columns=['Name', 'Area'])

# Save the DataFrame to a CSV file
df.sort_values(by=['Name']).to_csv('artistDataset.csv', index=False)
print("CSV file created successfully.")
'''
''''''

''

In [8]:
# Sau đó chạy cell này
def getSongInfo(name, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        links = soup.select(".play-this-track-playlinks .play-this-track-playlink")
        result = {}
        for link in links:
            result[link.getText().strip()] = link['href']
        return result
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}
def crawlAllSongOfArtist(artist):
    try:
        allSongsList = []
        artistQuery = artist.replace("+", "%252B").replace(" ", "+")
        url = f"https://www.last.fm/music/{artistQuery}/+tracks?date_preset=ALL"
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        pages = soup.find_all(attrs={"class": "pagination-page"})
        last_page = int(pages[-1].find('a').get_text().strip()) if len(pages) else 1
        for i in range(1, last_page+1):
        # For test using: 
        #for i in range(1, 2):
            res = requests.get(f"{url}&page={i}")
            res.raise_for_status()
            pg = res.text
            soup = BeautifulSoup(pg, 'html.parser')
            songs = soup.select("tr.chartlist-row > .chartlist-name > a")
            for song in songs:
            # For test using: 
            #for song in songs[:5]:
                songName = song.getText()
                songUrl = song['href']
                allSongsList.append([songName, getSongInfo(songName, f"https://www.last.fm{songUrl}")])
                print(f"artist: {artist} - page: {i}/{last_page} - song: {songName}")
                clear_output(wait=True)
        return allSongsList
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

https://api.socialcounts.org/youtube-video-live-view-count/8xg3vE8Ie_E